<a href="https://colab.research.google.com/github/Naaastenka/Project1/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline

In [ ]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [ ]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [ ]:
# Выделение целевой переменной и признаков
X = data.drop('MEDV', axis=1)
y = data['MEDV']

# Разделение данных на обучающую и тестовую выборки в отношении 80%/20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [ ]:
# Обучение стандартной линейной регрессии
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)

# Обучение Ridge регрессии
ridge = Ridge(random_state=RANDOM_STATE)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_ridge)

# Обучение Lasso регрессии
lasso = Lasso(random_state=RANDOM_STATE)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_lasso)

# Сохранение результатов
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

# Вывод результатов
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [ ]:
# 1. Подбор коэффициентов регуляризации с помощью GridSearchCV

param_grid = {'alpha': np.logspace(-5, 5, 11)}

# Ridge
ridge = Ridge()
ridge_cv = GridSearchCV(ridge, param_grid, cv=5)
ridge_cv.fit(X_train, y_train)
best_ridge = ridge_cv.best_estimator_
y_pred_ridge = best_ridge.predict(X_test)
r2_ridge_grid = r2_score(y_test, y_pred_ridge)

# Lasso
lasso = Lasso(max_iter=10000)
lasso_cv = GridSearchCV(lasso, param_grid, cv=5)
lasso_cv.fit(X_train, y_train)
best_lasso = lasso_cv.best_estimator_
y_pred_lasso = best_lasso.predict(X_test)
r2_lasso_grid = r2_score(y_test, y_pred_lasso)

# 2. Подбор коэффициентов регуляризации с помощью RidgeCV и LassoCV

# RidgeCV
ridge_cv_auto = RidgeCV(alphas=np.logspace(-5, 5, 11), cv=5)
ridge_cv_auto.fit(X_train, y_train)
y_pred_ridge_cv = ridge_cv_auto.predict(X_test)
r2_ridge_cv = r2_score(y_test, y_pred_ridge_cv)

# LassoCV
lasso_cv_auto = LassoCV(alphas=np.logspace(-5, 5, 11), cv=5, max_iter=10000)
lasso_cv_auto.fit(X_train, y_train)
y_pred_lasso_cv = lasso_cv_auto.predict(X_test)
r2_lasso_cv = r2_score(y_test, y_pred_lasso_cv)

# Вывод результатов
results_regression.loc[3] = ['Ridge (GridSearchCV)', 'task3', r2_ridge_grid]
results_regression.loc[4] = ['Lasso (GridSearchCV)', 'task3', r2_lasso_grid]
results_regression.loc[5] = ['Ridge (RidgeCV)', 'task3', r2_ridge_cv]
results_regression.loc[6] = ['Lasso (LassoCV)', 'task3', r2_lasso_cv]

print(results_regression)

                  model   task        R2
0                    LR  task2  0.668483
1                 Ridge  task2  0.665961
2                 Lasso  task2  0.666869
3  Ridge (GridSearchCV)  task3  0.668483
4  Lasso (GridSearchCV)  task3  0.668483
5       Ridge (RidgeCV)  task3  0.668483
6       Lasso (LassoCV)  task3  0.668483


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
# 1. Использование StandardScaler с Ridge и Lasso

# Ridge с StandardScaler
ridge_pipeline_std = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])
ridge_pipeline_std.fit(X_train, y_train)
y_pred_ridge_std = ridge_pipeline_std.predict(X_test)
r2_ridge_std = r2_score(y_test, y_pred_ridge_std)

# Lasso с StandardScaler
lasso_pipeline_std = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso())
])
lasso_pipeline_std.fit(X_train, y_train)
y_pred_lasso_std = lasso_pipeline_std.predict(X_test)
r2_lasso_std = r2_score(y_test, y_pred_lasso_std)

# 2. Использование MinMaxScaler с Ridge и Lasso

# Ridge с MinMaxScaler
ridge_pipeline_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('ridge', Ridge())
])
ridge_pipeline_minmax.fit(X_train, y_train)
y_pred_ridge_minmax = ridge_pipeline_minmax.predict(X_test)
r2_ridge_minmax = r2_score(y_test, y_pred_ridge_minmax)

# Lasso с MinMaxScaler
lasso_pipeline_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('lasso', Lasso())
])
lasso_pipeline_minmax.fit(X_train, y_train)
y_pred_lasso_minmax = lasso_pipeline_minmax.predict(X_test)
r2_lasso_minmax = r2_score(y_test, y_pred_lasso_minmax)

# Сохранение результатов

r2_ridge_standart_scaler = 0
r2_ridge_min_max_scaler = 0
r2_lasso_standart_scaler = 0
r2_lasso_min_max_scaler = 0
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

# Вывод результатов
print(results_regression)

                   model   task        R2
0                     LR  task2  0.668483
1                  Ridge  task2  0.665961
2                  Lasso  task2  0.666869
3   Ridge (GridSearchCV)  task3  0.668483
4   Lasso (GridSearchCV)  task3  0.668483
5        Ridge (RidgeCV)  task3  0.668483
6        Lasso (LassoCV)  task3  0.668483
7   Ridge_StandardScaler  task4  0.000000
8     Ridge_MinMaxScaler  task4  0.000000
9   Lasso_StandardScaler  task4  0.000000
10    Lasso_MinMaxScaler  task4  0.000000


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:

# Диапазон значений alpha для GridSearchCV
param_grid = np.logspace(-5, 5, 11)

# 1. Ridge с StandardScaler и GridSearchCV

ridge_pipeline_std = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

ridge_cv_std = GridSearchCV(ridge_pipeline_std, param_grid={'ridge__alpha': param_grid}, cv=5)
ridge_cv_std.fit(X_train, y_train)
y_pred_ridge_std = ridge_cv_std.best_estimator_.predict(X_test)
r2_ridge_standart_scaler_cv = r2_score(y_test, y_pred_ridge_std)

# 2. Ridge с MinMaxScaler и GridSearchCV

ridge_pipeline_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('ridge', Ridge())
])

ridge_cv_minmax = GridSearchCV(ridge_pipeline_minmax, param_grid={'ridge__alpha': param_grid}, cv=5)
ridge_cv_minmax.fit(X_train, y_train)
y_pred_ridge_minmax = ridge_cv_minmax.best_estimator_.predict(X_test)
r2_ridge_min_max_scaler_cv = r2_score(y_test, y_pred_ridge_minmax)

# 3. Lasso с StandardScaler и GridSearchCV

lasso_pipeline_std = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(max_iter=10000))
])

lasso_cv_std = GridSearchCV(lasso_pipeline_std, param_grid={'lasso__alpha': param_grid}, cv=5)
lasso_cv_std.fit(X_train, y_train)
y_pred_lasso_std = lasso_cv_std.best_estimator_.predict(X_test)
r2_lasso_standart_scaler_cv = r2_score(y_test, y_pred_lasso_std)

# 4. Lasso с MinMaxScaler и GridSearchCV

lasso_pipeline_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('lasso', Lasso(max_iter=10000))
])

lasso_cv_minmax = GridSearchCV(lasso_pipeline_minmax, param_grid={'lasso__alpha': param_grid}, cv=5)
lasso_cv_minmax.fit(X_train, y_train)
y_pred_lasso_minmax = lasso_cv_minmax.best_estimator_.predict(X_test)
r2_lasso_min_max_scaler_cv = r2_score(y_test, y_pred_lasso_minmax)

# Сохранение результатов
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

# Вывод результатов
print(results_regression)

                      model   task        R2
0                        LR  task2  0.668483
1                     Ridge  task2  0.665961
2                     Lasso  task2  0.666869
3      Ridge (GridSearchCV)  task3  0.668483
4      Lasso (GridSearchCV)  task3  0.668483
5           Ridge (RidgeCV)  task3  0.668483
6           Lasso (LassoCV)  task3  0.668483
7      Ridge_StandardScaler  task4  0.000000
8        Ridge_MinMaxScaler  task4  0.000000
9      Lasso_StandardScaler  task4  0.000000
10       Lasso_MinMaxScaler  task4  0.000000
11  Ridge_StandardScaler_CV  task5  0.668190
12    Ridge_MinMaxScaler_CV  task5  0.669765
13  Lasso_StandardScaler_CV  task5  0.668482
14    Lasso_MinMaxScaler_CV  task5  0.668484


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [ ]:
# 1. Ridge с StandardScaler и PolynomialFeatures

ridge_pipeline_std_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge())
])

ridge_pipeline_std_poly.fit(X_train, y_train)
y_pred_ridge_std_poly = ridge_pipeline_std_poly.predict(X_test)
r2_ridge_standart_scaler_poly = r2_score(y_test, y_pred_ridge_std_poly)

# 2. Ridge с MinMaxScaler и PolynomialFeatures

ridge_pipeline_minmax_poly = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge())
])

ridge_pipeline_minmax_poly.fit(X_train, y_train)
y_pred_ridge_minmax_poly = ridge_pipeline_minmax_poly.predict(X_test)
r2_ridge_min_max_scaler_poly = r2_score(y_test, y_pred_ridge_minmax_poly)

# 3. Lasso с StandardScaler и PolynomialFeatures

lasso_pipeline_std_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lasso', Lasso(max_iter=10000))
])

lasso_pipeline_std_poly.fit(X_train, y_train)
y_pred_lasso_std_poly = lasso_pipeline_std_poly.predict(X_test)
r2_lasso_standart_scaler_poly = r2_score(y_test, y_pred_lasso_std_poly)

# 4. Lasso с MinMaxScaler и PolynomialFeatures

lasso_pipeline_minmax_poly = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lasso', Lasso(max_iter=10000))
])

lasso_pipeline_minmax_poly.fit(X_train, y_train)
y_pred_lasso_minmax_poly = lasso_pipeline_minmax_poly.predict(X_test)
r2_lasso_min_max_scaler_poly = r2_score(y_test, y_pred_lasso_minmax_poly)

# Сохранение результатов
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

# Вывод результатов
print(results_regression)

                        model   task        R2
0                          LR  task2  0.668483
1                       Ridge  task2  0.665961
2                       Lasso  task2  0.666869
3        Ridge (GridSearchCV)  task3  0.668483
4        Lasso (GridSearchCV)  task3  0.668483
5             Ridge (RidgeCV)  task3  0.668483
6             Lasso (LassoCV)  task3  0.668483
7        Ridge_StandardScaler  task4  0.000000
8          Ridge_MinMaxScaler  task4  0.000000
9        Lasso_StandardScaler  task4  0.000000
10         Lasso_MinMaxScaler  task4  0.000000
11    Ridge_StandardScaler_CV  task5  0.668190
12      Ridge_MinMaxScaler_CV  task5  0.669765
13    Lasso_StandardScaler_CV  task5  0.668482
14      Lasso_MinMaxScaler_CV  task5  0.668484
15  Ridge_StandardScaler_Poly  task6  0.817136
16    Ridge_MinMaxScaler_Poly  task6  0.829886
17  Lasso_StandardScaler_Poly  task6  0.732274
18    Lasso_MinMaxScaler_Poly  task6  0.261126


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
# Диапазон значений alpha для GridSearchCV
param_grid = np.logspace(-5, 5, 11)
# 1. Ridge с StandardScaler, PolynomialFeatures и GridSearchCV

ridge_pipeline_std_poly_cv = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge())
])

ridge_cv_std_poly = GridSearchCV(ridge_pipeline_std_poly_cv, param_grid={'ridge__alpha': param_grid}, cv=5)
ridge_cv_std_poly.fit(X_train, y_train)
y_pred_ridge_std_poly_cv = ridge_cv_std_poly.best_estimator_.predict(X_test)
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, y_pred_ridge_std_poly_cv)

# 2. Ridge с MinMaxScaler, PolynomialFeatures и GridSearchCV

ridge_pipeline_minmax_poly_cv = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge())
])

ridge_cv_minmax_poly = GridSearchCV(ridge_pipeline_minmax_poly_cv, param_grid={'ridge__alpha': param_grid}, cv=5)
ridge_cv_minmax_poly.fit(X_train, y_train)
y_pred_ridge_minmax_poly_cv = ridge_cv_minmax_poly.best_estimator_.predict(X_test)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, y_pred_ridge_minmax_poly_cv)

# 3. Lasso с StandardScaler, PolynomialFeatures и GridSearchCV

lasso_pipeline_std_poly_cv = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lasso', Lasso(max_iter=10000))
])

lasso_cv_std_poly = GridSearchCV(lasso_pipeline_std_poly_cv, param_grid={'lasso__alpha': param_grid}, cv=5)
lasso_cv_std_poly.fit(X_train, y_train)
y_pred_lasso_std_poly_cv = lasso_cv_std_poly.best_estimator_.predict(X_test)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, y_pred_lasso_std_poly_cv)

# 4. Lasso с MinMaxScaler, PolynomialFeatures и GridSearchCV

lasso_pipeline_minmax_poly_cv = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lasso', Lasso(max_iter=10000))
])

lasso_cv_minmax_poly = GridSearchCV(lasso_pipeline_minmax_poly_cv, param_grid={'lasso__alpha': param_grid}, cv=5)
lasso_cv_minmax_poly.fit(X_train, y_train)
y_pred_lasso_minmax_poly_cv = lasso_cv_minmax_poly.best_estimator_.predict(X_test)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, y_pred_lasso_minmax_poly_cv)

# Сохранение результатов
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

# Вывод результатов
print(results_regression)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.846e+02, tolerance: 2.882e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.782e+02, tolerance: 2.764e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.870e+02, tolerance: 2.710e

                           model   task        R2
0                             LR  task2  0.668483
1                          Ridge  task2  0.665961
2                          Lasso  task2  0.666869
3           Ridge (GridSearchCV)  task3  0.668483
4           Lasso (GridSearchCV)  task3  0.668483
5                Ridge (RidgeCV)  task3  0.668483
6                Lasso (LassoCV)  task3  0.668483
7           Ridge_StandardScaler  task4  0.000000
8             Ridge_MinMaxScaler  task4  0.000000
9           Lasso_StandardScaler  task4  0.000000
10            Lasso_MinMaxScaler  task4  0.000000
11       Ridge_StandardScaler_CV  task5  0.668190
12         Ridge_MinMaxScaler_CV  task5  0.669765
13       Lasso_StandardScaler_CV  task5  0.668482
14         Lasso_MinMaxScaler_CV  task5  0.668484
15     Ridge_StandardScaler_Poly  task6  0.817136
16       Ridge_MinMaxScaler_Poly  task6  0.829886
17     Lasso_StandardScaler_Poly  task6  0.732274
18       Lasso_MinMaxScaler_Poly  task6  0.261126


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.085e+01, tolerance: 3.510e+00
  model = cd_fast.enet_coordinate_descent(


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:

# Определение параметров для GridSearchCV
param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'poly__degree': [1, 2, 3],  # Подбор степени полинома
    'model': [Ridge(), Lasso(max_iter=10000)],
    'model__alpha': np.logspace(-5, 5, 11)  # Коэффициент регуляризации
}

# Создание пайплайна
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Масштабирование
    ('poly', PolynomialFeatures(include_bias=False)),  # Полиномиальные признаки
    ('model', Ridge())  # Модель
])

# Подбор параметров с помощью GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Лучшая модель и её параметры
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
r2_best_model = r2_score(y_test, best_model.predict(X_test))

# Сохранение результатов
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

# Вывод параметров лучшей модели и её R2
print('Параметры лучшей модели:\n', best_params)
print('R2 лучшей модели:', r2_best_model)
print(results_regression)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.846e+02, tolerance: 2.882e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.782e+02, tolerance: 2.764e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.870e+02, tolerance: 2.710e

Параметры лучшей модели:
 {'model': Ridge(), 'model__alpha': 0.1, 'poly__degree': 3, 'scaler': MinMaxScaler()}
R2 лучшей модели: 0.8592608138177867
                           model   task        R2
0                             LR  task2  0.668483
1                          Ridge  task2  0.665961
2                          Lasso  task2  0.666869
3           Ridge (GridSearchCV)  task3  0.668483
4           Lasso (GridSearchCV)  task3  0.668483
5                Ridge (RidgeCV)  task3  0.668483
6                Lasso (LassoCV)  task3  0.668483
7           Ridge_StandardScaler  task4  0.000000
8             Ridge_MinMaxScaler  task4  0.000000
9           Lasso_StandardScaler  task4  0.000000
10            Lasso_MinMaxScaler  task4  0.000000
11       Ridge_StandardScaler_CV  task5  0.668190
12         Ridge_MinMaxScaler_CV  task5  0.669765
13       Lasso_StandardScaler_CV  task5  0.668482
14         Lasso_MinMaxScaler_CV  task5  0.668484
15     Ridge_StandardScaler_Poly  task6  0.817136
16

In [ ]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge (GridSearchCV),task3,0.668483
4,Lasso (GridSearchCV),task3,0.668483
5,Ridge (RidgeCV),task3,0.668483
6,Lasso (LassoCV),task3,0.668483
7,Ridge_StandardScaler,task4,0.000000
8,Ridge_MinMaxScaler,task4,0.000000
9,Lasso_StandardScaler,task4,0.000000


http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
data = pd.read_csv('adult.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [ ]:
# Разделим выборку на признаки и целевую переменную
X = data.drop('class', axis=1)  # Признаки
y = data['class']  # Целевая переменная

# Заменим значения целевой переменной на числовые
y = y.replace({'<=50K': 1, '>50K': 0})

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Определяем самый частый класс в целевой переменной
most_frequent_class = y.mode()[0]

# Создаем предсказания, где все значения равны самому частому классу
y_pred = [most_frequent_class] * len(y)

# Считаем метрики accuracy и f1_score
acc_most_frequent = accuracy_score(y, y_pred)
f1_most_frequent = f1_score(y, y_pred)

# Формируем результат
results_classification = pd.DataFrame(columns=['Model', 'Task', 'F1 Score', 'Accuracy'])
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

results_classification

,Model,Task,F1 Score,Accuracy
0,Most Frequent class,task10,0.8641,0.760718


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:
from sklearn.impute import SimpleImputer

# Проверим наличие пропусков
missing_data = data.isnull().sum()

# Если есть пропуски, заполним их самыми частыми значениями
if missing_data.any():
    imputer = SimpleImputer(strategy='most_frequent')
    data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)
else:
    data_imputed = data.copy()

# Проверим, остались ли пропуски после заполнения
missing_after_imputation = data_imputed.isnull().sum()

missing_data, missing_after_imputation

(age               0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 class             0
 dtype: int64,
 age               0
 workclass         0
 fnlwgt            0
 education         0
 education-num     0
 marital-status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital-gain      0
 capital-loss      0
 hours-per-week    0
 native-country    0
 class             0
 dtype: int64)

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [ ]:
# Выделим колонки с числовыми и категориальными переменными
numeric_columns = data_imputed.select_dtypes(include=['number']).columns.tolist()
categorical_columns = data_imputed.select_dtypes(include=['object']).columns.tolist()

numeric_columns, categorical_columns

(['age',
  'fnlwgt',
  'education-num',
  'capital-gain',
  'capital-loss',
  'hours-per-week'],
 ['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex',
  'native-country',
  'class'])

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# Определяем преобразование для числовых и категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns[:-1])  # исключаем 'class'
    ]
)

# Определяем алгоритмы
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearSVC': LinearSVC(max_iter=10000)
}

# Список для хранения результатов
results = []

# Применяем пайплайн и считаем cross_val_score для каждого алгоритма
for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])

    acc = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy').mean()
    f1 = cross_val_score(pipeline, X, y, cv=5, scoring='f1').mean()

    results.append([name, 'task13', f1, acc])

# Заполним DataFrame с результатами
for i, (name, task, f1, acc) in enumerate(results, start=1):
    results_classification.loc[i] = [name, task, f1, acc]

results_classification

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 194, in _get_response_values
    y_pred = prediction_method(X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 514, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py",

KeyboardInterrupt: 

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# Заменим значения '?' на пропуски и затем на самые частые значения
data.replace('?', pd.NA, inplace=True)

# Используем SimpleImputer для замены пропусков на самые частые значения
imputer = SimpleImputer(strategy='most_frequent')
data_imputed = pd.DataFrame(imputer.fit_transform(data), columns=data.columns)

# Обновляем признаки и целевую переменную
X_imputed = data_imputed.drop('class', axis=1)
y_imputed = data_imputed['class'].replace({'<=50K': 1, '>50K': 0})

# Выделим колонки с числовыми и категориальными переменными
numeric_columns = X_imputed.select_dtypes(include=['number']).columns.tolist()
categorical_columns = X_imputed.select_dtypes(include=['object']).columns.tolist()

# Обновляем преобразования для числовых и категориальных признаков
preprocessor_impute = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# Обновляем пайплайн и считаем cross_val_score для каждого алгоритма с учетом заполненных пропусков
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'LinearSVC': LinearSVC(max_iter=10000)
}

results_impute = []

for name, model in models.items():
    pipeline_impute = Pipeline(steps=[('preprocessor', preprocessor_impute), ('classifier', model)])

    acc = cross_val_score(pipeline_impute, X_imputed, y_imputed, cv=5, scoring='accuracy').mean()
    f1 = cross_val_score(pipeline_impute, X_imputed, y_imputed, cv=5, scoring='f1').mean()

    results_impute.append([f'{name}_impute', 'task14', f1, acc])

# Заполним DataFrame с результатами
results_classification = pd.DataFrame(columns=['Model', 'Task', 'F1 Score', 'Accuracy'])
for i, (name, task, f1, acc) in enumerate(results_impute, start=4):
    results_classification.loc[i] = [name, task, f1, acc]

results_classification


TypeError: boolean value of NA is ambiguous

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification